In [3]:
from llama_index.core import VectorStoreIndex, get_response_synthesizer,load_index_from_storage,StorageContext,Document
from llama_index.core.retrievers import VectorIndexRetriever
from llama_index.core.query_engine import RetrieverQueryEngine
from llama_index.core.response_synthesizers.type import ResponseMode
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.llms.openai_like import OpenAILike
from transformers import AutoTokenizer
from llama_index.core import Settings
from llama_index.core.node_parser import SimpleFileNodeParser,TokenTextSplitter
from pathlib import Path
from llama_index.readers.file import FlatReader
from pythainlp.tokenize import sent_tokenize

embed_model = HuggingFaceEmbedding(model_name="intfloat/multilingual-e5-large",device='cpu')

tokenizer = AutoTokenizer.from_pretrained("SeaLLMs/SeaLLM-7B-v2").encode
custom_api = 'float16-56hBpTpGA5VYSs13Awe9U1FjGaTo5pNODx67cnI066EBP5rewr'
custom_url = 'https://api.float16.cloud/v1/llamaindex/'
llm = OpenAILike(api_base=custom_url,
                 api_key=custom_api,
                 max_tokens=512,
                 model='SeaLLM-7b-v2',
                 api_version='v1',
                 )
embed_model = HuggingFaceEmbedding(model_name="intfloat/multilingual-e5-large",device='cpu')

token_spliter = TokenTextSplitter(
    chunk_size=512,
    chunk_overlap=64,
    tokenizer=tokenizer,
)

load_index = load_index_from_storage(StorageContext.from_defaults(persist_dir='./localstore/'),embed_model=embed_model)

[Document(id_='413325e1-5eac-4c9c-9611-f6682fd3d219', embedding=None, metadata={'filename': 'demo.txt', 'extension': '.txt'}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, text='เครื่องสักการะเครื่องบูชาใครไปนั่งตรงนั้นก็ได้รับการสักการะบูชา เพราะว่าผู้แสดงธรรมเป็นผู้ที่ควรแก่สักการะบูชา ที่เรียกว่าอุเทสิกเจดีย์เค้าเรียกว่าเป็นสิ่งที่ควรค่า ไม่ได้ว่าจะเป็นเฉพาะพระภิกษุนะฆราวาสเองก็ตาม ถ้าเขาจัดตั้งการบูชาไว้ก็คือต้องการบูชาอุเทสิกเจดีย์\nอุเทสิกเจดีย์ตัวนี้ไม่มีวัตถุปรากฏ แต่เป็นสิ่งที่เนื่องด้วยพระตถาคต คำว่าสิ่งที่เนื่องด้วยพระตถาคตคือการแสดงธรรม เพราะอุเทสิกมีความหมายว่าการยกขึ้น แสดงอะไรก็ตามที่เป็นการยกขึ้น แสดงขึ้นมาให้ได้ทราบถึงความเป็นพระสัมมาสัมพุทธะ ทางด้านไตรรัตน์ไตรลักษณ์ ก็คือพระรัตนตรัยเนี่ย พระพุทธ พระธรรม พระสงฆ์ ก็ถือว่าเป็นการบูชาพระรัตนตรัยนั่นเอง เป็นการบูชาพระธรรมด้วย บูชาพระรัตนตรัยด้วยแล้วก็บูชาบุคคลผู้แสดงธรรมด้วย ก็ได้ทั้งหมดในการกระทำการบูชา\nแต่ทีนี้ข้อมูลของคำว่าอุเทสิกเจดีย์ถูกเปลี่ยนแปลงในระบบการศึกษาไทย ก็อุเทสิกเจดีย์เ

In [ ]:
retrieval_engine = load_index.as_retriever(similarity_top_k = 1)
nodes = token_spliter.get_nodes_from_documents(docs)
responseEmbed = retrieval_engine.retrieve("")
OutEmbed = responseEmbed[0].text
f = open("demo.txt", "w", encoding='utf8')
f.write(OutEmbed)
f.close()
docs = FlatReader().load_data(Path("./demo.txt"))
print(docs)
nodes = token_spliter.get_nodes_from_documents(docs)
#nodes = token_spliter.get_nodes_from_documents(text_parser.get_nodes_from_documents(docs)) 
print(nodes)

In [4]:
Settings.embed_model = embed_model
Settings.llm = llm
Settings.context_window = 4000 
Settings.tokenizer = tokenizer
Settings.num_output = 512
response_synthesizer = get_response_synthesizer(
      response_mode="simple_summarize" 
)
index = VectorStoreIndex(nodes)
print(index)
query_engine = index.as_query_engine(
    llm=llm,
    response_synthesizer = response_synthesizer)

responseLLm = query_engine.query("<summarization_query_thai>")

In [5]:
print(responseLLm)

อุเทสิกเจดีย์เป็นสิ่งที่เนื่องด้วยพระตถาคต ไม่ใช่วัตถุใดๆ ที่ปรากฏให้เห็น แต่เป็นสิ่งที่เนื่องด้วยพระตถาคตที่แสดงธรรมหรือการยกขึ้นของสิ่งใดสิ่งหนึ่งให้เห็นถึงความเป็นพระสัมมาสัมพุทธะ ในทางไตรรัตน์ไตรลักษณ์ อุเทสิกเจดีย์หมายถึงการบูชาพระรัตนตรัย ซึ่งรวมถึงพระพุทธ พระธรรม และพระสงฆ์ นอกจากนี้ยังรวมถึงการบูชาบุคคลผู้แสดงธรรมด้วย การบูชาอุเทสิกเจดีย์เป็นการบูชาพระรัตนตรัยและการบูชาพระธรรม ซึ่งเป็นการบูชาพระพุทธเจ้าและการแสดงธรรม
